In [1]:
pip install elasticsearch

     |████████████████████████████████| 477 kB 2.5 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 1.4 MB/s  eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

     |████████████████████████████████| 13.1 MB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 8.7 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 7.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

import os
import json
from elasticsearch import Elasticsearch

#### Spark

In [4]:
# Creamos una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("practica_ibd").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/18 10:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Leemos los archivos JSON en un DataFrame
archivos_json = spark.read.option("multiline", "true").json("/opt/workspace")

# Seleccionamos los campos interesantes
campos_seleccionados = archivos_json.select(
    archivos_json['title'].alias('title'),
    archivos_json['author_name'].getItem(0).alias('author_name'),
    archivos_json['first_publish_year'].alias('publish_year'),
    archivos_json['language'].alias('language'),
    archivos_json['subject'].alias('subject'),
    archivos_json['number_of_pages_median'].alias('num_pages'),
    archivos_json['id_amazon'].alias('id_amazon'),
    archivos_json['ratings_average'].alias('ratings')
)

In [ ]:
# Se supone que facilita ciertas cosas en spark crear un id pero ns para qué
#from pyspark.sql.functions import monotonically_increasing_id

# Añadir una columna de identificador único al DataFrame
#campos_seleccionados = campos_seleccionados.withColumn("id", monotonically_increasing_id())

#campos_seleccionados.show()

In [6]:
campos_seleccionados.count()

2399

In [7]:
# Convertimos el Dataframe a Pandas para facilitar la inserción en elasticsearch
campos_pandas = campos_seleccionados.toPandas()
campos_pandas[:100]

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"[fre, ita, por, ger, chi, rus, spa, eng, dut]","[Frankenstein (Fictitious character), Frankens...",240.0,"[B08M25T3PW, B093XMVLXF, B000LQ3XU2, 198604261...",3.900585
1,A Christmas Carol,Charles Dickens,1843.0,"[chi, spa, rus, eng, kor, fre, und, ita, fin, ...","[Ghost stories, Readers, Ebenzer Scrooge (Fict...",114.0,"[B002D6DY1G, B000QNQ8J8, , B01N40IX3W, B0022WF...",3.883117
2,The Picture of Dorian Gray,Oscar Wilde,1890.0,"[dut, chi, heb, arm, ukr, spa, fre, rus, cze, ...",[British and irish fiction (fictional works by...,253.0,"[B000OKWFA4, B008MWZR1A, B00ABMJJSQ, B00XF4RG4...",4.041666
3,Kalender 2021,Paula Paulsen,2020.0,[ger],None,120.0,None,NaN
4,Northanger Abbey,Jane Austen,1818.0,"[por, ger, mul, spa, heb, chi, pol, und, ita, ...","[Mate selection, Cousins, Fiction, Young women...",254.0,"[B000H8TULK, B0012BZNYE, B000HMBM7Q, B0017HFQN...",3.785714
...,...,...,...,...,...,...,...,...
95,"Full Dark, No Stars",Stephen King,2010.0,"[dut, chi, spa, fre, rus, pol, ger, eng]","[Chevrolet Suburbans, 1955 doubled die cents, ...",427.0,"[1444712551, 2226239936, 1501197940, 189295053...",4.071429
96,If There Be Thorns,V. C. Andrews,1981.0,"[und, dut, pol, ger, fre, rus, spa, eng]","[series:dollanganger, Novel, half-brothers, ba...",374.0,"[1476799466, 0671415085, 0671606859, , B01F9FR...",3.500000
97,The Haunted Hotel,Wilkie Collins,1878.0,"[eng, fre, spa]","[Fiction, English fiction, Hotels, Horror fict...",216.0,"[069271345X, 1985276585, 1983593230, 849682919...",3.000000
98,The Drawing of the Three,Stephen King,1987.0,"[ger, spa, chi, rus, eng, fre]","[sociopaths, dissociative identity disorder, c...",455.0,"[B0026L4HTG, 2738206751, 0451210859, , 1471232...",4.166666


In [8]:
# Buscamos las filas que son nulas completas
filas_con_nulos = campos_pandas[campos_pandas.isnull().all(axis=1)]
print(filas_con_nulos)

    title author_name  publish_year language subject  num_pages id_amazon  \
27   None        None           NaN     None    None        NaN      None   
511  None        None           NaN     None    None        NaN      None   

     ratings  
27       NaN  
511      NaN  


In [9]:
indices_eliminar = filas_con_nulos.index.tolist()
indices_eliminar

[27, 511]

In [10]:
campos_pandas.drop(indices_eliminar, inplace=True)

In [11]:
campos_pandas

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"[fre, ita, por, ger, chi, rus, spa, eng, dut]","[Frankenstein (Fictitious character), Frankens...",240.0,"[B08M25T3PW, B093XMVLXF, B000LQ3XU2, 198604261...",3.900585
1,A Christmas Carol,Charles Dickens,1843.0,"[chi, spa, rus, eng, kor, fre, und, ita, fin, ...","[Ghost stories, Readers, Ebenzer Scrooge (Fict...",114.0,"[B002D6DY1G, B000QNQ8J8, , B01N40IX3W, B0022WF...",3.883117
2,The Picture of Dorian Gray,Oscar Wilde,1890.0,"[dut, chi, heb, arm, ukr, spa, fre, rus, cze, ...",[British and irish fiction (fictional works by...,253.0,"[B000OKWFA4, B008MWZR1A, B00ABMJJSQ, B00XF4RG4...",4.041666
3,Kalender 2021,Paula Paulsen,2020.0,[ger],None,120.0,None,NaN
4,Northanger Abbey,Jane Austen,1818.0,"[por, ger, mul, spa, heb, chi, pol, und, ita, ...","[Mate selection, Cousins, Fiction, Young women...",254.0,"[B000H8TULK, B0012BZNYE, B000HMBM7Q, B0017HFQN...",3.785714
...,...,...,...,...,...,...,...,...
2394,Awful Horror Stories,None,NaN,None,None,NaN,[B000GJ5WEY],NaN
2395,Besessen. Roman. ( Horror).,None,NaN,None,None,NaN,None,NaN
2396,Gaslit Horror,Hugh Lamb,NaN,None,None,NaN,None,NaN
2397,HORROR-7.,None,NaN,None,None,NaN,[B0010TAAGE],NaN


In [12]:
spark.stop()

#### Elasticsearch

In [13]:
client = Elasticsearch("http://elasticsearch:9200")

In [16]:
# Iterar sobre cada fila del Pandas DataFrame
for index, row in campos_pandas.iterrows():
    # Convertir la fila a un objeto JSON
    json_obj = row.to_json()
    
    # Insertar el objeto JSON en Elasticsearch
    client.index(index='my_index', body=json_obj)

In [17]:
# Realizar una consulta para recuperar algunos documentos
res = client.search(index='my_index', size=5)  

# Mostrar los resultados
for hit in res['hits']['hits']:
    print(hit['_source'])

{'title': 'Frankenstein or The Modern Prometheus', 'author_name': 'Mary Wollstonecraft Shelley', 'publish_year': 1818.0, 'language': ['fre', 'ita', 'por', 'ger', 'chi', 'rus', 'spa', 'eng', 'dut'], 'subject': ['Frankenstein (Fictitious character)', "Frankenstein's monster (Fictitious character)", 'Fiction', 'Victor Frankenstein (Fictitious character)', 'Scientists', 'Monsters', 'Fiction, horror', 'Frankenstein (fictitious character), fiction', 'Physicians, fiction', 'British fiction (fictional works by one author)', 'Scientists, fiction', 'Geneva (Switzerland), fiction', "Children's fiction", 'Horror stories', 'Fiction, science fiction, general', 'Fiction, general', 'Horror tales', 'Frankenstein (Shelley, Mary Wollstonecraft)', 'Large type books', 'English fiction', 'Study guides', 'Examinations', 'Criticism and interpretation', 'Fiction, gothic', 'English language, textbooks for foreign speakers', 'Reading comprehension', 'Monsters, fiction', 'English literature', 'Comics & graphic no

In [18]:
# Obtener información del índice
index_info = client.cat.indices(index="my_index", format='json')

# Comprobamos el número de documentos
for info in index_info:
    if info['index'] == "my_index":
        doc_count = info['docs.count']
        print(f"El número de documentos en el índice  es: {doc_count}")

El número de documentos en el índice  es: 2397


In [15]:
# Verificar si el índice existe antes de intentar eliminarlo
if client.indices.exists(index='my_index'):
    # Eliminar el índice
    client.indices.delete(index='my_index')
    print(f"El índice my_index ha sido eliminado.")
else:
    print(f"El índice my_index no existe.")

El índice my_index ha sido eliminado.
